In [ ]:
!pip install crewAI

In [ ]:
!pip install langchain_openai

In [ ]:
from google.colab import userdata
import os
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
    # Then set it for the OpenAI library
import openai
#openai.api_key = openai_api_key
client = openai.OpenAI(api_key=openai_api_key)

In [ ]:
from crewai import Agent

# Define a basic agent
general_agent = Agent(
    role='Movie  Agent',
    goal='Find Theaters to watch Movies requested by user with description',
    backstory='Need to find theaters where movies are screened.You can access information from https://in.bookmyshow.com/ and https://ticketnew.com/movies for schedule'
)

In [ ]:
from crewai import Task

# Define a task for the agent
planning_task = Task(
    description='Find Theaters Near Cognizant Siruseri Chennai office where Tamil Movies are running currently',
    expected_output='List of Theaters and address with Movie Name,Next earliest Date where movie can be watched',
    agent=general_agent
)

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI
# Assemble the crew
crew = Crew(
    agents=[general_agent],
    tasks=[planning_task],
    process=Process.sequential,
    verbose=True
)
llm = ChatOpenAI(model="gpt-4-turbo-preview")


In [ ]:
result = crew.kickoff()

# Display the final output
print(result)

In [ ]:
!pip install 'crewai[tools]' tavily-python

In [ ]:
from crewai_tools import TavilySearchTool

    # To search only on specific domains
tavily_tool_include = TavilySearchTool(
        include_domains=["https://in.bookmyshow.com/,https://ticketnew.com/movies/chennai"]
    )

    # To exclude specific domains from the search
tavily_tool_exclude = TavilySearchTool(
        exclude_domains=["spam-site.net", "old-blog.info"]
    )

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import TavilySearchTool
import os

os.environ["TAVILY_API_KEY"] = ""


    # To search only on specific domains
tavily_tool_include = TavilySearchTool(
        include_domains=["https://in.bookmyshow.com/,https://ticketnew.com/movies/chennai"]
    )

    # To exclude specific domains from the search
tavily_tool_exclude = TavilySearchTool(
        exclude_domains=["spam-site.net", "old-blog.info"]
    )

MovieAgent = Agent(
        role='Theaters Researcher',
        goal='Find Currently Running Movies in Theaters.',
        backstory='Need to find theaters where movies are screened.You can access information from https://in.bookmyshow.com/ and https://ticketnew.com/movies for schedule.',
        tools=[tavily_tool_include], # Use the tool configured with include_domains
        verbose=True
    )

SearchTask = Task(
        description='Find Theaters Near Cognizant Siruseri Chennai office where Tamil Movies are running currently.',
        expected_output='List of Theaters and address with Movie Name,Review and Rating if possible ,Next earliest Date where movie can be watched in a CSV format',
        agent=MovieAgent
    )

crew = Crew(
        agents=[MovieAgent],
        tasks=[SearchTask],
        verbose=False
    )
result = crew.kickoff()
print(result)

In [ ]:
!pip install autogen
!pip install autogen-agentchat~=0.2
!pip install pyautogen

In [ ]:
import os
import autogen
import autogen_core
import autogen_agentchat
from autogen import AssistantAgent, UserProxyAgent

llm_config = {"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}
assistant = AssistantAgent("assistant", llm_config=llm_config)

user_proxy = UserProxyAgent(
    "user_proxy", code_execution_config={"executor": autogen.coding.LocalCommandLineCodeExecutor(work_dir="coding")}
)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Find Theaters Near Cognizant Siruseri Chennai office where Tamil Movies are running currently.",
)

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeElementFromWebsiteTool


In [ ]:
element_scraper = ScrapeElementFromWebsiteTool()

In [ ]:
data_extractor = Agent(
    role='Data Extractor',
    goal='Extract movie names,theater name, theater address',
    backstory='Focused expert who knows how to retrieve precise data from HTML.',
    tools=[element_scraper]
)

In [ ]:
data_extraction_task = Task(
    description=(
        "1. Visit the page: https://ticketnew.com/movies/chennai"
        "2. Search for <h5> and </h5> and extract the text between it and this most likely the movie title"
        "3. Extract URLS like https://ticketnew.com/movies "
        "4. Get the list of theaters and address from above"
        "5. Share the information in bulletted list format"
    ),
    expected_output="Movie Name,Theater Address.",
    agent=data_extractor,
)

In [ ]:
from crewai import Crew, Process

# (Code to define agents and tasks goes here)

my_crew = Crew(
    agents=[data_extractor],
    tasks=[data_extraction_task],
    process=Process.sequential,
)

# Kick off the crew with inputs
result = my_crew.kickoff(inputs={'news_url': 'https://ticketnew.com/movies/chennai'})

# Print the result
print(result)

Maximum iterations reached. Requesting final answer.


╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): - Movie Name: Idli Kadai, Theater: ABC Cinemas, Address: 123 Main St, Chennai
- Movie Name: Kantara: A Legend Chapter-1, Theater: XYZ Theatres, Address: 456 High St, Chennai
- Movie Name: They Call Him OG, Theater: PQR Multiplex, Address: 789 Park Ave, Chennai
- Movie Name: Kiss, Theater: STU Cinema, Address: 321 River Rd, Chennai
- Movie Name: Shakthi Thirumagan, Theater: MNO Theaters, Address: 654 Elm St, Chennai
- Movie Name: Tron: Ares, Theater: JKL Theatres, Address: 987 Birch St, Chennai
- Movie Name: Lokah Chapter 1: Chandra, Theater: GHI Cinemas, Address: 135 Maple St, Chennai
- Movie Name: Marutham, Theater: DEF Theatres, Address: 246 Pine St, Chennai
N
